In [89]:
import sys
import os
import torch
current_dir = os.getcwd()
sys.path.append(os.path.join(current_dir, '..'))
from model.model_config import ModelConfig
from data.data_config import DataConfig

model_config = ModelConfig("qwen", "D://LLM//CodeT5", torch_dtype=torch.float32)
model, tokenizer = model_config.load_model()
data_config = DataConfig("D://Study//NPR&RL//ProjectV1//assets//Train", tokenizer, is_test=True)
dataset = data_config.load_dataset()

TypeError: ModelConfig.__init__() got an unexpected keyword argument 'torch_dtype'

In [13]:
data = dataset[0]
print(data.keys())
input_ids = data["input_ids"]
input_mask = data["input_mask"]
target_ids = data["target_ids"]
target_mask = data["target_mask"]

print(input_ids.shape)
print(input_mask.shape)
print(target_ids.shape)
print(target_mask.shape)


dict_keys(['idx', 'bug_method', 'bug_line', 'fix_line', 'prompt', 'input_ids', 'target_ids', 'input_mask', 'target_mask'])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 256])
torch.Size([1, 256])


In [16]:

logits = model(input_ids, input_mask, target_ids, target_mask).logits
print(logits.shape)

torch.Size([1, 256, 32100])


In [94]:
model = model.float()
output = model.generate(
        input_ids = data["input_ids"], 
        attention_mask = data["input_ids"].ne(tokenizer.pad_token_id),
        do_sample=False,
        top_p=1,
        top_k=10,
        temperature=1.5,
        max_length=32,
        return_dict_in_generate=True,
        output_scores=True
    )

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [95]:
import torch
print(output.sequences.squeeze(0).shape)
print(torch.stack(output.scores, dim=0).squeeze(1).shape)
scores = torch.stack(output.scores, dim=0).squeeze(1)
print(scores)
logits = scores.softmax(dim=-1)
token = output.sequences.squeeze(0)[:-1]
token_logits = logits[torch.arange(token.shape[0]), token]
log_logits = torch.log(token_logits).sum().item()
print(log_logits)
print(torch.isnan(token_logits).any())
print(torch.isinf(token_logits).any())


output_txt = tokenizer.decode(
    token, skip_special_tokens=True
)
print(output_txt)

torch.Size([32])
torch.Size([31, 32100])
tensor([[ 1.0951e+01,  2.8392e+01,  2.7427e+00,  ...,  4.1671e+00,
          4.0522e+00,  9.0531e+00],
        [ 4.2390e+00,  8.5767e+00,  4.1810e+00,  ...,  7.3698e+00,
          6.6297e+00,  2.8217e+01],
        [ 9.5582e-03,  7.2327e+00,  2.6452e+00,  ...,  2.5483e+00,
          2.7126e+00,  2.1610e+00],
        ...,
        [-1.3057e+00,  4.0206e+00,  7.8687e+00,  ...,  3.4302e+00,
          6.5345e+00,  3.2062e-01],
        [-1.1365e+00,  1.0944e+00,  7.5901e+00,  ...,  3.1522e+00,
          6.5545e+00,  1.1913e-01],
        [-1.0851e+00,  1.3962e+00,  6.9470e+00,  ...,  3.6295e+00,
          6.4179e+00,  1.9753e-01]])
-295.63360595703125
tensor(False)
tensor(False)
() { if (i.getpasswordstate()){puffer =; } else {(kpw) {
